# Setup

### Import necessary modules and do some basic setup.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay, precision_score, recall_score, roc_auc_score, roc_curve
from sklearn.utils import class_weight

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers, regularizers

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Dotenv
from dotenv import dotenv_values

# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_unet import *
from utils.utils_resnet import *
from utils.utils_plot import *

### Define some paths and constants.

In [ ]:
config = dotenv_values(".env")

# Paths
PATH_ERA5 = config['PATH_ERA5']
PATH_EOBS = config['PATH_EOBS']

# Some constants
G = 9.80665

# Options
DATE_START = '1979-01-01'
DATE_END = '2020-12-31'
YY_TRAIN = [1979, 2015]
YY_TEST = [2016, 2021]
LEVELS = [500, 850, 1000]
LONS_INPUT = [-25, 30]
LATS_INPUT = [30, 75]
LONS_PREC = [-25, 30]
LATS_PREC = [30, 75]
BATCH_SIZE = 64
PRECIP_DATA = 'ERA5-low' # Options: ERA5-hi, ERA5-low, E-OBS
PRECIP_XTRM = 0.95 # Percentile (threshold) for the extremes

# Data preparation

## Target variable: precipitation field

In [ ]:
# Load precipitation
if PRECIP_DATA == 'ERA5-hi':
    pr = get_nc_data(PATH_ERA5 + '/precipitation/orig_grid/daily/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.tp
elif PRECIP_DATA == 'ERA5-low':
    pr = get_nc_data(PATH_ERA5 + '/precipitation/day_grid1/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.tp
elif PRECIP_DATA in ['E-OBS', 'EOBS']:
    pr = get_nc_data(PATH_EOBS + '/*nc', DATE_START, DATE_END, LONS_PREC, LATS_PREC)
    pr = pr.rr
    pr = pr.coarsen(lon=5, lat=5, boundary="trim").mean()
    pr = pr.fillna(0) # Over the seas. Not optimal...
else:
    raise ValueError('Precipitation data not well defined')

In [ ]:
# Add a dimension to be used as channel in the DNN
pr = pr.expand_dims('level', -1)
pr

In [ ]:
# Compute the extreme exceedence
qq = xr.DataArray(pr).quantile(PRECIP_XTRM, dim='time')
pr_xtrm = xr.DataArray(pr > qq)
pr_xtrm = pr_xtrm*1 # Transform to number

In [ ]:
# Extract coordinates for precip
lats_y = pr.lat.to_numpy()
lons_y = pr.lon.to_numpy()

In [ ]:
# Get the date with the max number of grid points with pr > threshold
x = pr_xtrm.sum(dim=['lat', 'lon'])
plot_date = x.argmax(dim='time')

# Plot precip, threshold exceedance and threshold values
fig, axs = plt.subplots(1, 3, figsize=(21,4))

plot_map(axs[0], lons_y, lats_y, pr[plot_date,:,:].to_numpy().squeeze(), title="Daily precipitation", cmap=mpl.cm.YlGnBu)
plot_map(axs[1], lons_y, lats_y, pr_xtrm[plot_date,:,:].to_numpy().squeeze(), title="Threshold exceedance", cmap=mpl.cm.YlGnBu)
plot_map(axs[2], lons_y, lats_y, qq.to_numpy().squeeze(), title="Threshold values", cmap=mpl.cm.YlGnBu)

## Input data: meteorological fields

In [ ]:
# Load geopotential height
z = get_era5_data(PATH_ERA5 + '/geopotential/grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
z = z.sel(level=LEVELS)

# Get Z in geopotential height (m)
z.z.values = z.z.values/G

# Load temperature
t2m = get_era5_data(PATH_ERA5 + '/temperature/grid1/Grid1_*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
t2m['time'] = pd.DatetimeIndex(t2m.time.dt.date)
t2m = t2m.rename_vars({'T2MMEAN': 't'})

# Load relative humidity
rh = get_era5_data(PATH_ERA5 + '/relative_humidity/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
rh['time'] = pd.DatetimeIndex(rh.time.dt.date)
rh = rh.sel(level=LEVELS)

# Load total column water
tcw = get_era5_data(PATH_ERA5 + '/total_column_water/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
tcw['time'] = pd.DatetimeIndex(tcw.time.dt.date)

# Load wind components
u850 = get_era5_data(PATH_ERA5 + '/U_wind/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
u850['time'] = pd.DatetimeIndex(u850.time.dt.date)
v850 = get_era5_data(PATH_ERA5 + '/V_wind/day_grid1/*.nc', DATE_START, DATE_END, LONS_INPUT, LATS_INPUT)
v850['time'] = pd.DatetimeIndex(v850.time.dt.date)

# Checking dimensions
print('dimension of pr:', pr.dims)
print('dimension of z', z.dims)
print('dimension of t2m:', t2m.dims)
print('dimension of rh:', rh.dims)
print('dimension of tcw:', tcw.dims)
print('dimension of u:', u850.dims)
print('dimension of v:', v850.dims)


In [ ]:
# Merge arrays
X = xr.merge([z, t2m, rh, tcw, u850, v850])
X

In [ ]:
# Invert lat axis if needed
if X.lat[0].values < X.lat[1].values:
    X = X.reindex(lat=list(reversed(X.lat)))
    
# Get axes
lats_x = X.lat
lons_x = X.lon

### Split data and transform

In [ ]:
# Split into training and test
X_train_full = X.sel(time=slice(f'{YY_TRAIN[0]}-01-01', f'{YY_TRAIN[1]}-12-31'))
X_test = X.sel(time=slice(f'{YY_TEST[0]}-01-01', f'{YY_TEST[1]}-12-31'))

pr_train_full = pr.sel(time=slice(f'{YY_TRAIN[0]}-01-01', f'{YY_TRAIN[1]}-12-31'))
pr_test = pr.sel(time=slice(f'{YY_TEST[0]}-01-01', f'{YY_TEST[1]}-12-31'))

pr_xtrm_train_full = pr_xtrm.sel(time=slice(f'{YY_TRAIN[0]}-01-01', f'{YY_TRAIN[1]}-12-31'))
pr_xtrm_test = pr_xtrm.sel(time=slice(f'{YY_TEST[0]}-01-01', f'{YY_TEST[1]}-12-31'))

In [ ]:
# Create a data generator
dic = {'z': LEVELS,
       't': None,
       'r': LEVELS,
       'tcwv': None,
       'u': None,
       'v': None}

from utils.utils_ml import *

YY_VALID = 2005

dg_train = DataGeneratorWithExtremes(X_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     pr_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     pr_xtrm_train_full.sel(time=slice(f'{YY_TRAIN[0]}', f'{YY_VALID}')),
                                     dic, batch_size=BATCH_SIZE, load=True)
dg_valid = DataGeneratorWithExtremes(X_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     pr_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     pr_xtrm_train_full.sel(time=slice(f'{YY_VALID+1}', f'{YY_TRAIN[1]}')),
                                     dic, mean=dg_train.mean, std=dg_train.std,
                                     batch_size=BATCH_SIZE, load=True)
dg_test = DataGeneratorWithExtremes(X_test, pr_test, pr_xtrm_test, dic,
                                    mean=dg_train.mean, std=dg_train.std,
                                    batch_size=BATCH_SIZE, load=True, shuffle=False)


In [ ]:
i_shape = dg_train.X.shape[1:]
o_shape = dg_train.y.shape[1:]

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')

In [ ]:
# Plotting the mean precipitation
fig = plt.figure(figsize=(7,5))
ax = fig.add_subplot(1, 1, 1)
vals = np.mean(dg_train.y, axis=0).squeeze()
plot_map(ax, lons_y, lats_y, vals, title="Mean precipitation", cmap=mpl.cm.YlGnBu)

In [ ]:
# Plotting the mean of the predictors
n_figs = len(dg_train.X[0,0,0,:])
ncols = 5
nrows = -(-n_figs // ncols)
fig, axes = plt.subplots(figsize=(24, 3.2*nrows), ncols=ncols, nrows=nrows)
for i in range(n_figs):
    i_row = i // ncols
    i_col = i % ncols
    ax = axes[i_row, i_col]
    vals = np.mean(dg_train.X[:,:,:,i], axis=0).to_numpy()
    plot_map(ax, lons_x, lats_x, vals, title=f"Average of feature {i+1}")

# Model creation

In [ ]:
class DeepFactory(tf.keras.Model):
    """
    Model factory.
    """

    def __init__(self, arch, input_size, output_size, for_extremes=False, latent_dim=128, dropout_rate=0.2, 
                 use_batch_norm=True, inner_activation='relu', output_scaling=1, output_crop=None):
        super(DeepFactory, self).__init__()
        self.arch = arch
        self.input_size = list(input_size)
        self.output_size = list(output_size)
        self.for_extremes = for_extremes
        self.latent_dim = latent_dim
        self.dropout_rate = dropout_rate
        self.use_batch_norm = use_batch_norm
        self.inner_activation = inner_activation
        self.output_scaling = output_scaling
        self.output_crop = output_crop
        
        self.last_activation = 'relu'
        if for_extremes:
            self.last_activation = 'sigmoid'

        if arch == 'Davenport-2021':
            self.build_Davenport_2021()
        elif arch == 'CNN-2L':
            self.build_CNN_2L()
        elif arch == 'Unet':
            self.build_Unet()
        elif arch == 'Pan-2019':
            self.build_Pan_2019()
        elif arch =='Conv-LTSM':
            self.build_convLTSM()
        else:
            raise ValueError('The architecture was not correctly defined')
        
        
    def build_Davenport_2021(self):
        """
        Based on: Davenport, F. V., & Diffenbaugh, N. S. (2021). Using Machine Learning to 
        Analyze Physical Causes of Climate Change: A Case Study of U.S. Midwest Extreme Precipitation. 
        Geophysical Research Letters, 48(15). https://doi.org/10.1029/2021GL093787
        """
        
        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        x = layers.Conv2D(16, 3, padding='same', activity_regularizer=regularizers.l2(0.01))(inputs)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.SpatialDropout2D(self.dropout_rate)(x) # In original: simple Dropout
        x = layers.Conv2D(16, 3, padding='same', activity_regularizer=regularizers.l2(0.01))(x)
        x = layers.Activation('relu')(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.SpatialDropout2D(self.dropout_rate)(x) # In original: simple Dropout
        x = layers.Flatten()(x)                
        x = layers.Dense(self.latent_dim, activity_regularizer=regularizers.l2(0.001))(x) # In original: 16
        x = layers.Activation('relu')(x)

        next_shape = self.get_shape_for(stride_factor=4)

        # Upsampling. In original: no decoder
        x = self.dense_block(x, np.prod(next_shape))
        x = layers.Reshape(target_shape=next_shape)(x)
        x = self.deconv_block(x, 16, 3, stride=2)
        x = self.deconv_block(x, 16, 3, stride=2)
        x = self.conv_block(x, 1, 3, activation=self.last_activation)
        outputs = self.final_cropping_block(x)
 
        self.model = keras.Model(inputs, outputs, name="Davenport-2021")
        
        
    def build_Pan_2019(self):
        """
        Based on: Pan, B., Hsu, K., AghaKouchak, A., & Sorooshian, S. (2019). 
        Improving Precipitation Estimation Using Convolutional Neural Network. 
        Water Resources Research, 55(3), 2301–2321. https://doi.org/10.1029/2018WR024090
        """
        # In original: padding='valid'
        
        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        x = layers.Conv2D(15, 4, padding='same')(inputs)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Conv2D(20, 4, padding='same')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.Conv2D(20, 4, padding='same')(x)
        x = layers.Activation('relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.MaxPooling2D(pool_size=2)(x)
        x = layers.Flatten()(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(self.latent_dim, activation='relu')(x) # In original: 60

        next_shape = self.get_shape_for(stride_factor=4)

        # Upsampling. In original: no decoder
        x = self.dense_block(x, np.prod(next_shape))
        x = layers.Reshape(target_shape=next_shape)(x)
        x = self.deconv_block(x, 20, 4, stride=2)
        x = self.deconv_block(x, 20, 4, stride=2)
        x = self.conv_block(x, 15, 4)
        x = self.conv_block(x, 1, 3, activation=self.last_activation)
        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="Pan-2019")


    def build_CNN_2L(self):

        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        x = self.conv_block(inputs, 32, 3, stride=2, with_batchnorm=True, with_dropout=True)
        x = self.conv_block(x, 64, 3, stride=2, with_batchnorm=True, with_dropout=True)
        x = layers.Flatten()(x)
        x = self.dense_block(x, self.latent_dim, activation='sigmoid')

        next_shape = self.get_shape_for(stride_factor=4)

        # Upsampling
        x = self.dense_block(x, np.prod(next_shape))
        x = layers.Reshape(target_shape=next_shape)(x)
        x = self.deconv_block(x, 64, 3, stride=2)
        x = self.deconv_block(x, 32, 3, stride=2)
        x = self.deconv_block(x, 1, 3, activation=self.last_activation)
        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="CNN-v1")
        

    def build_convLTSM(self):
        
        
        
        inputs = layers.Input(shape=self.input_size)
        # the order of the extended_shape must be after the input layer
        expanded_shape = self.input_size
        expanded_shape.insert(0, 1)
        
        x = layers.Reshape(target_shape=expanded_shape)(inputs)
        
        x = layers.ConvLSTM2D(filters=32, kernel_size=(3, 3),
                                         return_sequences=True,
                                         padding = 'same',
                                         go_backwards=True,
                                         activation='tanh', recurrent_activation='hard_sigmoid',
                                         data_format = 'channels_last',
                                         kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                                         dropout=0.4, recurrent_dropout=0.2
                                         )(x)
        x = layers.BatchNormalization()(x)

        x = layers.ConvLSTM2D(filters=16, kernel_size=(3, 3),
                                         return_sequences=True,
                                         go_backwards=True,
                                         padding = 'same',
                                         activation='tanh', recurrent_activation='hard_sigmoid',
                                         kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                                         data_format = 'channels_last',
                                         dropout=0.4, recurrent_dropout=0.2
                                         )(x)
        x = layers.BatchNormalization()(x)

        x = layers.ConvLSTM2D(filters=8, kernel_size=(3, 3),
                                         return_sequences=False,
                                         go_backwards=True,
                                         padding = 'same',
                                         activation='tanh', recurrent_activation='hard_sigmoid',
                                         kernel_initializer='glorot_uniform', unit_forget_bias=True, 
                                         data_format = 'channels_last',
                                         dropout=0.3, recurrent_dropout=0.2
                                         )(x)
        x = layers.BatchNormalization()(x)

        x = layers.Conv2D(filters=16, kernel_size=(1, 1),
                           activation='relu',
                           data_format='channels_last')(x)


        x = layers.MaxPooling2D(pool_size=(2, 2), padding='same')(x)
        #model.add(Flatten())
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.25)(x)

        x = layers.Conv2DTranspose(filters=1, kernel_size=(1, 1), strides=(2, 2), 
                           activation='sigmoid',padding='same',
                           data_format='channels_last')(x)

        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="Conv-LTSM")
        
        
    def build_Unet(self):
        """
        Based on: U-Net: https://github.com/nikhilroxtomar/Unet-for-Person-Segmentation/blob/main/model.py
        """
        
        filters_nb = 64
        
        # Downsampling
        inputs = layers.Input(shape=self.input_size)
        
        # Pad if necessary
        x = self.padding_block(inputs, factor=16)
        
        s1, p1 = self.unet_encoder_block(x, filters_nb)
        s2, p2 = self.unet_encoder_block(p1, filters_nb * 2)
        s3, p3 = self.unet_encoder_block(p2, filters_nb * 4)
        s4, p4 = self.unet_encoder_block(p3, filters_nb * 8)
        
        x = self.conv_block(p4, filters_nb * 16, 3, initializer='he_normal', with_batchnorm=True, with_dropout=True)
        b1 = self.conv_block(x, filters_nb * 16, 3, initializer='he_normal', with_batchnorm=True)

        # Upsampling
        d1 = self.unet_decoder_block(b1, s4, filters_nb * 8)
        d2 = self.unet_decoder_block(d1, s3, filters_nb * 4)
        d3 = self.unet_decoder_block(d2, s2, filters_nb * 2)
        d4 = self.unet_decoder_block(d3, s1, filters_nb, is_last=True)
        
        # Additional upsampling for downscaling
        x = self.handle_output_scaling(d4)

        x = self.conv_block(x, 1, 1, activation=self.last_activation)
        outputs = self.final_cropping_block(x)

        self.model = keras.Model(inputs, outputs, name="U-Net-v1")
        
        
    def unet_encoder_block(self, input, filters, kernel_size=3):
        x = self.conv_block(input, filters, kernel_size, initializer='he_normal', with_batchnorm=True, with_dropout=True)
        x = self.conv_block(x, filters, kernel_size, initializer='he_normal', with_batchnorm=True)
        p = layers.MaxPooling2D((2, 2))(x)
        
        return x, p

    
    def unet_decoder_block(self, input, skip_features, filters, conv_kernel_size=3, deconv_kernel_size=2, is_last=False):
        x = self.deconv_block(input, filters, deconv_kernel_size, stride=2)
        x = layers.Concatenate()([x, skip_features])
        x = self.conv_block(x, filters, conv_kernel_size, initializer='he_normal', with_batchnorm=True, with_dropout=True)
        x = self.conv_block(x, filters, conv_kernel_size, initializer='he_normal', with_batchnorm=(not is_last))

        return x
        
        
    def conv_block(self, input, filters, kernel_size=3, stride=1, padding='same', initializer='default', activation='default', 
                   with_batchnorm=False, with_pooling=False, with_dropout=False, with_late_activation=False):
        if activation == 'default':
            activation = self.inner_activation
            
        conv_activation = activation
        if with_late_activation:
            conv_activation = None
            
        if initializer == 'default':
            x = layers.Conv2D(filters, kernel_size, strides=(stride, stride), padding=padding, activation=conv_activation)(input)
        else:
            x = layers.Conv2D(filters, kernel_size, strides=(stride, stride), padding=padding, activation=conv_activation, kernel_initializer=initializer)(input)
            
        if with_batchnorm:
            x = layers.BatchNormalization()(x)
        if with_late_activation:
            x = layers.Activation(activation)(x)
        if with_pooling:
            x = layers.MaxPooling2D(pool_size=2)(x)
        if with_dropout:
            x = layers.SpatialDropout2D(self.dropout_rate)(x)
        
        return x
    
    
    def deconv_block(self, input, filters, kernel_size=3, stride=1, padding='same', initializer='default', activation='default', 
                     with_batchnorm=False, with_dropout=False):
        if activation == 'default':
            activation = self.inner_activation
        
        if initializer == 'default':
            x = layers.Conv2DTranspose(filters, kernel_size, strides=stride, padding=padding, activation=activation)(input)
        else:
            x = layers.Conv2DTranspose(filters, kernel_size, strides=stride, padding=padding, activation=activation, kernel_initializer=initializer)(input)
            
        if with_batchnorm:
            x = layers.BatchNormalization()(x)
        if with_dropout:
            x = layers.SpatialDropout2D(self.dropout_rate)(x)
        
        return x
    
    
    def dense_block(self, input, units, activation='default', with_dropout=False):
        if activation == 'default':
            activation=self.inner_activation
            
        x = layers.Dense(units, activation=activation)(input)
        if with_dropout:
            x = layers.Dropout(self.dropout_rate)(x)
            
        return x

    
    def handle_output_scaling(self, input, with_batchnorm=False):
        if self.output_scaling > 1:
            if self.output_scaling == 2:
                x = self.deconv_block(input, 64, 3, stride=2, with_batchnorm=with_batchnorm)
            elif self.output_scaling == 3:
                x = self.deconv_block(input, 64, 3, stride=3, with_batchnorm=with_batchnorm)
            elif self.output_scaling == 4:
                x = self.deconv_block(input, 64, 3, stride=2, with_batchnorm=with_batchnorm)
                x = self.deconv_block(x, 64, 3, stride=2, with_batchnorm=with_batchnorm)
            elif self.output_scaling == 5:
                x = self.deconv_block(input, 64, 3, stride=3, with_batchnorm=with_batchnorm)
                x = self.deconv_block(x, 64, 3, stride=2, with_batchnorm=with_batchnorm)
            else:
                raise NotImplementedError('Level of downscaling not implemented')
        else:
            x = input
        
        if self.output_crop:
            raise NotImplementedError('Manual cropping not yet implemented')
            
        return x
            
        
    def padding_block(self, x, factor):
        h, w = x.get_shape().as_list()[1:3]
        dh = 0
        dw = 0
        if h % factor > 0:
            dh = factor - h % factor
        if w % factor > 0:
            dw = factor - w % factor
        if dh > 0 or dw > 0:
            top_pad = dh//2
            bottom_pad = dh//2 + dh%2
            left_pad = dw//2
            right_pad = dw//2 + dw%2
            x = layers.ZeroPadding2D(padding=((top_pad, bottom_pad), (left_pad, right_pad)))(x)
        
        return x
        
        
    def final_cropping_block(self, x):
        # Compute difference between reconstructed width and hight and the desired output size.
        h, w = x.get_shape().as_list()[1:3]
        h_tgt, w_tgt = self.output_size[:2]
        dh = h - h_tgt
        dw = w - w_tgt

        if dh < 0 or dw < 0:
            raise ValueError(f'Negative values in output cropping dh={dh} and dw={dw}')

        # Add to decoder cropping layer and final reshaping
        x = layers.Cropping2D(cropping=((dh//2, dh-dh//2), (dw//2, dw-dw//2)))(x)
        #x = layers.Reshape(target_shape=self.output_size,)(x)
        
        return x
        

    def get_shape_for(self, stride_factor):
        next_shape = self.output_size.copy()
        next_shape[0] = int(np.ceil(next_shape[0]/stride_factor))
        next_shape[1] = int(np.ceil(next_shape[1]/stride_factor))

        return next_shape

        
    def call(self, x):
        return self.model(x)
    


In [ ]:
# Computing the necessary output scaling.
dlons_x = float(lons_x[1] - lons_x[0])
dlats_x = float(lats_x[0] - lats_x[1])
dlons_y = float(lons_y[1] - lons_y[0])
dlats_y = float(lats_y[0] - lats_y[1])

output_scaling = int(dlons_x / dlons_y)
output_crop = None

In [ ]:
def initiate_optimizer(lr_method, lr=.0004, init_lr=0.01, max_lr=0.01):
    if lr_method == 'Cyclical':
        # Cyclical learning rate
        steps_per_epoch = dg_train.n_samples // BATCH_SIZE
        clr = tfa.optimizers.CyclicalLearningRate(
            initial_learning_rate=init_lr,
            maximal_learning_rate=max_lr,
            scale_fn=lambda x: 1/(2.**(x-1)),
            step_size=2 * steps_per_epoch)
        optimizer = tf.keras.optimizers.Adam(clr)
    elif lr_method == 'CosineDecay':
        decay_steps = EPOCHS * (dg_train.n_samples / BATCH_SIZE)
        lr_decayed_fn = tf.keras.optimizers.schedules.CosineDecay(
            init_lr, decay_steps)
        optimizer = tf.keras.optimizers.Adam(lr_decayed_fn)
    elif lr_method == 'Constant':
        optimizer = tf.keras.optimizers.Adam(learning_rate = lr)
    else:
        raise ValueError('learning rate schedule not well defined.')
        
    return optimizer

In [ ]:
# Compute weights for the weighted binary crossentropy
weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(pr_xtrm.values),
    y=pr_xtrm.values.flatten()
)

print('Weights for the weighted binary crossentropy:')
print(f'Classes: {np.unique(pr_xtrm.values)}, weights: {weights}')

# Create loss function for the extremes
xtrm_loss = weighted_binary_cross_entropy(
    weights={0: weights[0].astype('float32'), 1: weights[1].astype('float32')})

In [ ]:
def analyze_predictions(y_pred, dg, qq, pred_xtrm=False, show_plots=True, plot_most_extreme=True, plot_worst_best=False, plot_scores=True, plot_confusion_components=True):
    if pred_xtrm:
        y_pred_bool = y_pred >= 0.5
    else:
        y_pred_bool = y_pred > qq.to_numpy().squeeze()
        
    # Multiply to transorm to numeric values
    y_pred_bool = y_pred_bool * 1
    
    # Extract true values
    y_xtrm = dg.y_xtrm.to_numpy().squeeze()
    y_prec = dg.y.to_numpy().squeeze()
    
    # Get the index of the max # of extremes
    i_max_obs = np.argmax(np.sum(y_xtrm, axis=(1,2)))
    
    if show_plots and plot_most_extreme:
        if pred_xtrm:
            fig, axes = plt.subplots(figsize=(12, 3.5), ncols=2, nrows=1)
            plot_map(axes[0], lons_y, lats_y, y_xtrm[i_max_obs], title="Day with max # extremes - truth (xtrm)", vmin=0, vmax=1)
            plot_map(axes[1], lons_y, lats_y, y_pred[i_max_obs], title="Day with max # extremes - prediction (prob xtrm)", vmin=0, vmax=1)
        else:
            fig, axes = plt.subplots(figsize=(24, 3.5), ncols=4, nrows=1)
            vmax = max(np.max(y_prec[i_max_obs]), np.max(y_pred[i_max_obs]))
            plot_map(axes[0], lons_y, lats_y, y_prec[i_max_obs], title="Day with max # extremes - truth (val)", vmin=0, vmax=vmax)
            plot_map(axes[1], lons_y, lats_y, y_pred[i_max_obs], title="Day with max # extremes - prediction (val)", vmin=0, vmax=vmax)
            plot_map(axes[2], lons_y, lats_y, y_xtrm[i_max_obs], title="Day with max # extremes - truth (xtrm)", vmin=0, vmax=1)
            plot_map(axes[3], lons_y, lats_y, y_pred_bool[i_max_obs], title="Day with max # extremes - prediction (xtrm)", vmin=0, vmax=1)
        
    # Get the index of the max/min difference between prediction and obs
    y_diffs_series = np.sum(np.absolute(y_xtrm - y_pred_bool), axis=(1,2))
    i_worst_pred = np.argmax(y_diffs_series)
    i_best_pred = np.argmin(y_diffs_series)
    
    if show_plots and plot_worst_best:
        if pred_xtrm:
            fig, axes = plt.subplots(figsize=(24, 3.5), ncols=4, nrows=1)
            plot_map(axes[0], lons_y, lats_y, y_xtrm[i_worst_pred], title="Worst prediction - truth", vmin=0, vmax=1)
            plot_map(axes[1], lons_y, lats_y, y_pred[i_worst_pred], title="Worst prediction - prediction", vmin=0, vmax=1)
            plot_map(axes[2], lons_y, lats_y, y_xtrm[i_best_pred], title="Best prediction - truth", vmin=0, vmax=1)
            plot_map(axes[3], lons_y, lats_y, y_pred[i_best_pred], title="Best prediction - prediction", vmin=0, vmax=1)
        else:
            fig, axes = plt.subplots(figsize=(24, 3.5), ncols=4, nrows=1)
            plot_map(axes[0], lons_y, lats_y, y_xtrm[i_worst_pred], title="Worst prediction - truth", vmin=0, vmax=1)
            plot_map(axes[1], lons_y, lats_y, y_pred_bool[i_worst_pred], title="Worst prediction - prediction", vmin=0, vmax=1)
            plot_map(axes[2], lons_y, lats_y, y_xtrm[i_best_pred], title="Best prediction - truth", vmin=0, vmax=1)
            plot_map(axes[3], lons_y, lats_y, y_pred_bool[i_best_pred], title="Best prediction - prediction", vmin=0, vmax=1)
    
    # Compute scores
    precision, recall = eval_confusion_matrix_scores_on_map(y_xtrm, y_pred_bool)

    if pred_xtrm:
        roc_auc = eval_roc_auc_score_on_map(y_xtrm, y_pred)
        
    if show_plots and plot_scores:
        if pred_xtrm:
            fig, axes = plt.subplots(figsize=(18, 4), ncols=3, nrows=1)
        else:
            fig, axes = plt.subplots(figsize=(12, 4), ncols=2, nrows=1)

        plot_map(axes[0], lons_y, lats_y, precision, title="Precision", vmin=0, vmax=1)
        plot_map(axes[1], lons_y, lats_y, recall, title="Recall", vmin=0, vmax=1)
        if pred_xtrm:
            plot_map(axes[2], lons_y, lats_y, roc_auc, title="ROC AUC", vmin=0.5, vmax=1)

    if show_plots and plot_confusion_components:
        tn, fp, fn, tp = eval_confusion_matrix_on_map(y_xtrm, y_pred_bool)
        fig, axes = plt.subplots(figsize=(24, 4), ncols=4, nrows=1)
        plot_map(axes[0], lons_y, lats_y, tn, title="True negative")
        plot_map(axes[1], lons_y, lats_y, fp, title="False positive")
        plot_map(axes[2], lons_y, lats_y, fn, title="False negative")
        plot_map(axes[3], lons_y, lats_y, tp, title="True positive")
    
    plt.show()
    
    if pred_xtrm:
        return np.mean(precision), np.mean(recall), np.mean(roc_auc)
    
    return np.mean(precision), np.mean(recall)

In [ ]:
# Define hyperparameters
EPOCHS = 100
LR_METHOD = 'Constant'  # Cyclical, CosineDecay, Constant
    
# Early stopping
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                                            restore_best_weights=True)
                                            
# Default model options
opt_model = {'latent_dim': 128,
             'dropout_rate': 0.2}

# Default training options
opt_training = {'epochs': EPOCHS,
                'callbacks': [callback]}

# Default optimizer options
opt_optimizer = {'lr_method': 'Constant',
                 'lr': 0.0004,
                 'init_lr': 0.01}

In [ ]:
models = {
          'Dav-orig': {'model': 'Davenport-2021', 'run': False,
                       'opt_model': {'latent_dim': 16},
                       'opt_optimizer': {'lr_method': 'Constant'}}, # original
          'Dav-64': {'model': 'Davenport-2021', 'run': False,
                     'opt_model': {'latent_dim': 64},
                     'opt_optimizer': {'lr_method': 'Constant'}},
          'CNN-2l': {'model': 'CNN-2L', 'run': False,
                     'opt_model': {'latent_dim': 64},
                     'opt_optimizer': {'lr_method': 'Constant'}},
          'UNET': {'model': 'Unet', 'run': False,
                   'opt_model': {'output_scaling': output_scaling, 'output_crop': output_crop},
                   'opt_optimizer': {'lr_method': 'CosineDecay'}},
          'Pan-orig': {'model': 'Pan-2019', 'run': False,
                       'opt_model': {'latent_dim': 60},
                       'opt_optimizer': {'lr_method': 'Constant', 'lr': 1e-4}},
          'Conv-LTSM': {'model': 'Conv-LTSM', 'run': True,
                        'opt_model': {'latent_dim': 64},
                        'opt_optimizer': {'lr_method': 'Constant'}},
         }

In [ ]:
# Predict the amount of precipitation
df_prec = pd.DataFrame(columns = ['id', 'name', 'n_params', 'opt_model', 'opt_optimizer',
                                  'train_pr_rmse', 'test_pr_rmse', 
                                  'train_xtrm_precision', 'test_xtrm_precision', 
                                  'train_xtrm_recall', 'test_xtrm_recall'])
df_xtrm = pd.DataFrame(columns = ['id', 'name', 'n_params', 'opt_model', 'opt_optimizer',
                                  'train_pr_rmse', 'test_pr_rmse', 
                                  'train_xtrm_precision', 'test_xtrm_precision', 
                                  'train_xtrm_recall', 'test_xtrm_recall', 
                                  'train_xtrm_roc_auc', 'test_xtrm_roc_auc'])

train_for_prec = True
train_for_xtrm = True

if train_for_prec:
        
    for m_id in models:
        # Clear session and set tf seed
        keras.backend.clear_session()
        tf.random.set_seed(42)
        
        if not models[m_id]['run']:
            continue

        # Extract model name and options
        model = models[m_id]['model']
        opt_model_i = models[m_id]['opt_model']
        opt_optimizer_i = models[m_id]['opt_optimizer']
        opt_model_new = opt_model.copy()
        opt_model_new.update(opt_model_i)
        opt_optimizer_new = opt_optimizer.copy()
        opt_optimizer_new.update(opt_optimizer_i)
        print(f'Running: {m_id} - {model} - {opt_model_i} - {opt_optimizer_i}')
        df_prec = df_prec.append({'id': m_id, 'name': model, 'opt_model': opt_model_i, 'opt_optimizer': opt_optimizer_i}, ignore_index=True)

        # Switch to precipitation values
        dg_train.for_extremes(False)
        dg_valid.for_extremes(False)
        dg_test.for_extremes(False)
        
        optimizer = initiate_optimizer(**opt_optimizer_new)

        # Create the model and compile
        m = DeepFactory(model, i_shape, o_shape, for_extremes=False, **opt_model_new)
        # Warning: When using regularizers, the loss function is the entire loss, ie (loss metrics) + (regularization term)!
        # But the loss displayed as part of the metrics, is only the loss metric. The regularization term is not added there. -> can be different!!
        m.compile(
            loss='mse', 
            metrics=['mse'], 
            optimizer=optimizer
        )
        print(f'Number of parameters: {m.model.count_params()}')
        df_prec.at[df_prec.index[-1], 'n_params'] = m.model.count_params()

        # Train
        hist = m.fit(dg_train, validation_data=dg_valid, verbose=0, **opt_training)
        
        # Predict and save scores
        y_pred_train = m.predict(dg_train.X.to_numpy()).squeeze()
        y_pred_test = m.predict(dg_test.X.to_numpy()).squeeze()
        df_prec.at[df_prec.index[-1], 'train_pr_rmse'] = np.sqrt(np.square(np.subtract(dg_train.y.to_numpy().squeeze(), y_pred_train)).mean())
        df_prec.at[df_prec.index[-1], 'test_pr_rmse'] =  np.sqrt(np.square(np.subtract(dg_test.y.to_numpy().squeeze(), y_pred_test)).mean())

        # Plot training evolution
        pd.DataFrame(hist.history).plot(figsize=(8, 5))
        ymin = 0.8 * min([min(hist.history['loss']), min(hist.history['val_loss']), min(hist.history['mse'])])
        ymax = min(max(max(hist.history['loss']), max(hist.history['val_loss'])), 
                   2 * max(sum(hist.history['loss'])/len(hist.history['loss']), sum(hist.history['val_loss'])/len(hist.history['val_loss'])))
        plt.ylim(ymin, ymax)
        plt.grid(True)
        plt.show()
        
        # Analyze predictions
        print('Plotting results of the training period.')
        precision, recall = analyze_predictions(y_pred_train, dg_train, qq, pred_xtrm=False, show_plots=False)
        df_prec.at[df_prec.index[-1], 'train_xtrm_precision'] = precision
        df_prec.at[df_prec.index[-1], 'train_xtrm_recall'] = recall
        plt.show()
        
        print('Plotting results of the testing period.')
        precision, recall = analyze_predictions(y_pred_test, dg_test, qq, pred_xtrm=False)
        df_prec.at[df_prec.index[-1], 'test_xtrm_precision'] = precision
        df_prec.at[df_prec.index[-1], 'test_xtrm_recall'] = recall
        plt.show()
    
        print(df_prec.iloc[-1])
        
        print(f"\n{'*' * 100}\n")
        
if train_for_xtrm:

    for m_id in models:
        # Clear session and set tf seed
        keras.backend.clear_session()
        tf.random.set_seed(42)

        if not models[m_id]['run']:
            continue
        
        # Extract model name and options
        model = models[m_id]['model']
        opt_model_i = models[m_id]['opt_model']
        opt_optimizer_i = models[m_id]['opt_optimizer']
        opt_model_new = opt_model.copy()
        opt_model_new.update(opt_model_i)
        opt_optimizer_new = opt_optimizer.copy()
        opt_optimizer_new.update(opt_optimizer_i)
        print(f'Running: {m_id} - {model} - {opt_model_i} - {opt_optimizer_i}')
        df_xtrm = df_xtrm.append({'id': m_id, 'name': model, 'opt_model': opt_model_i, 'opt_optimizer': opt_optimizer_i}, ignore_index=True)
        
        # Switch to precipitation extremes
        dg_train.for_extremes(True)
        dg_valid.for_extremes(True)
        dg_test.for_extremes(True)
        
        optimizer = initiate_optimizer(**opt_optimizer_new)

        # Create the model and compile
        m = DeepFactory(model, i_shape, o_shape, for_extremes=True, **opt_model_new)
        m.compile(
            loss=xtrm_loss,
            optimizer=optimizer
        )
        print(f'Number of parameters: {m.model.count_params()}')
        df_xtrm.at[df_xtrm.index[-1], 'n_params'] = m.model.count_params()

        # Train
        hist = m.fit(dg_train, validation_data=dg_valid, verbose=1, **opt_training)
        
        # Assess and save scores
        y_pred_train = m.predict(dg_train.X.to_numpy()).squeeze()
        y_pred_test = m.predict(dg_test.X.to_numpy()).squeeze()
    
        # Plot training evolution
        pd.DataFrame(hist.history).plot(figsize=(8, 5))
        ymin = 0.8 * min([min(hist.history['loss']), min(hist.history['val_loss'])])
        ymax = min(max(max(hist.history['loss']), max(hist.history['val_loss'])), 
                   2 * max(sum(hist.history['loss'])/len(hist.history['loss']), sum(hist.history['val_loss'])/len(hist.history['val_loss'])))
        plt.ylim(ymin, ymax)
        plt.grid(True)
        plt.show()
        
        # Analyze predictions
        print('Plotting results of the training period.')
        precision, recall, roc_auc = analyze_predictions(y_pred_train, dg_train, qq, pred_xtrm=True, show_plots=False)
        df_xtrm.at[df_xtrm.index[-1], 'train_xtrm_precision'] = precision
        df_xtrm.at[df_xtrm.index[-1], 'train_xtrm_recall'] = recall
        df_xtrm.at[df_xtrm.index[-1], 'train_xtrm_roc_auc'] = roc_auc
        plt.show()
        
        print('Plotting results of the testing period.')
        precision, recall, roc_auc = analyze_predictions(y_pred_test, dg_test, qq, pred_xtrm=True)
        df_xtrm.at[df_xtrm.index[-1], 'test_xtrm_precision'] = precision
        df_xtrm.at[df_xtrm.index[-1], 'test_xtrm_recall'] = recall
        df_xtrm.at[df_xtrm.index[-1], 'test_xtrm_roc_auc'] = roc_auc
        plt.show()

        print(df_xtrm.iloc[-1])
        
        print(f"\n{'*' * 100}\n")

In [ ]:
df_prec

In [ ]:
df_xtrm